In [2]:
!pip install haversine

import json
from haversine import haversine
from google.colab import files

# 위경도 거리 계산 함수
def calculate_distance(coord1, coord2):
    return haversine(coord1, coord2)

# 가장 가까운 값을 찾는 함수
def find_nearest_step_node(gate, step_nodes):
    gate_coords = (gate["latitude"], gate["longitude"])
    nearest_node = min(step_nodes, key=lambda x: calculate_distance(
        gate_coords, (x["position"]["latitude"], x["position"]["longitude"])))
    return nearest_node

In [3]:
# 파일 업로드
uploaded_gate = files.upload()
uploaded_step_node = files.upload()

# 업로드한 파일명을 확인하여 코드에 반영
gate_file_name = next(iter(uploaded_gate))
step_node_file_name = next(iter(uploaded_step_node))

# 모든 게이트에 대해 가장 가까운 노드 찾기
with open(gate_file_name, "r", encoding="utf-8") as f:
    gate_data = json.load(f)

with open(step_node_file_name, "r", encoding="utf-8") as f:
    step_node_data = json.load(f)

total_gates = len(gate_data)
print(f"Processing {total_gates} gates:")
nearest_list = []
for i, gate in enumerate(gate_data, start=1):
    nearest_node = find_nearest_step_node(gate, step_node_data)

    # 생성할 JSON 데이터 구성
    data = {
        "subway_gate_id": gate["id"],
        "nearest_step_node": {
            "id": nearest_node["id"],
            "crosswalk": nearest_node["crosswalk"],
            "position": {
                "latitude": nearest_node["position"]["latitude"],
                "longitude": nearest_node["position"]["longitude"]
            }
        }
    }

    nearest_list.append(data)

    # 진행 상황 출력
    print(f"\rProgress: {i}/{total_gates}", end="", flush=True)

Saving gate_location_in_seoul_ver2.json to gate_location_in_seoul_ver2 (1).json


Saving step_node.json to step_node (1).json
Processing 1963 gates:
Progress: 1963/1963

In [4]:
# 결과를 파일에 쓰기
output_file_name = "step_nearest_list.json"
with open(output_file_name, "w", encoding="utf-8") as f:
    json.dump(nearest_list, f, ensure_ascii=False, indent=2)

print(f"\nProcessing complete. Check '{output_file_name}' for the results.")


Processing complete. Check 'step_nearest_list.json' for the results.
